In [266]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import tensorflow as tf

In [219]:
# Download and save the archived data

url = 'http://mrtee.europa.renci.org/~bblanton/ANN/'
to = "../data"

def maybe_download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    print(os.path.join(to,filename))
    print(url+filename)
    if force or not os.path.exists(os.path.join(to,filename)):
        filename, _ = urlretrieve(url + filename, os.path.join(to,filename))
    statinfo = os.stat(os.path.join(to,filename))
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        raise Exception(
          'Failed to verify' + filename + '. Can you get to it with a browser?')
    return filename

data_filename = maybe_download('ann_dataset1.tar', 5642240)

../data/ann_dataset1.tar
http://mrtee.europa.renci.org/~bblanton/ANN/ann_dataset1.tar
Found and verified ann_dataset1.tar


In [220]:
# Extract files from the archive
def maybe_extract(filename, force=False):
    extract_folder = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    root = os.path.dirname(filename)
    if os.path.isdir(extract_folder) and not force:
    # You may override by setting force=True.
        print('%s already present - Skipping extraction of %s.' % (root, filename))
    else:
        print('Extracting data for %s. This may take a while. Please wait.' % root)
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall(path = root)
        tar.close()
    data_files = [
        os.path.join(extract_folder, d) for d in sorted(os.listdir(extract_folder))
        if os.path.isdir(extract_folder)]
    return data_files
  
data_filename = "../data/ann_dataset1.tar"
data_files = maybe_extract(data_filename)

../data already present - Skipping extraction of ../data/ann_dataset1.tar.


In [262]:
# Load files and produce dataset
def maybe_load(file_names):
    names = ('index','time', 'long', 'lat', 'param1', 'param2', 'param3', 'param4', 'out1', 'out2')
    datafile_length = 193
    dataset = np.ndarray(shape=(len(file_names), datafile_length, len(names)))
    for i in range(0,len(file_names)):
        a = np.loadtxt(file_names[i])
        a = np.asarray([x for xs in a for x in xs],dtype='d').reshape([datafile_length,len(names)])
        dataset[i,:,:] = a
        if i%100 == 0:
            print("Processed %d/%d \n"%(i,len(file_names)))
    return dataset

dataset = maybe_load(data_files)
print(dataset.shape)

Processed 0/324 

Processed 100/324 

Processed 200/324 

Processed 300/324 

(324, 193, 10)


In [263]:
# train, validation, and test dataset percentages
train_percent = 70
valid_percent = 15
test_percent = 15

# train, validation, and test dataset indices
# test: test_start : valid_start-1
# validation: valid_start : train_start-1
# training: train_start : dataset.shape[0]
test_start = 0 
valid_start = int(test_percent/100.0*dataset.shape[0])
train_start = int((test_percent+valid_percent)/100.0*dataset.shape[0])

# Shuffle file indices
file_indices = range(dataset.shape[0])
np.random.shuffle(file_indices)

# Assign datasets
test_dataset = np.array([dataset[j,:,:] for j in [file_indices[i] for i in range(test_start, valid_start)]])
valid_dataset = np.array([dataset[j,:,:] for j in [file_indices[i] for i in range(valid_start, train_start)]])
train_dataset = np.array([dataset[j,:,:] for j in [file_indices[i] for i in range(train_start, dataset.shape[0])]])

# Save memory
#del(dataset)
print("Test dataset: "+str(test_dataset.shape))
print("Validation dataset: "+str(valid_dataset.shape))
print("Training dataset: "+str(train_dataset.shape))

Test dataset: (48, 193, 10)
Validation dataset: (49, 193, 10)
Training dataset: (227, 193, 10)


In [448]:
print("Test dataset: "+str(test_dataset.shape))
print("Validation dataset: "+str(valid_dataset.shape))
print("Training dataset: "+str(train_dataset.shape))
print(train_output.shape)

Test dataset: (48, 193, 10)
Validation dataset: (49, 193, 10)
Training dataset: (227, 193, 10)
(43811, 1)


In [447]:
def accuracy_rmse(predictions, outputs):
    err = predictions-outputs
    return np.sqrt(np.mean(err*err))

In [427]:
# Reshape the data and normalize

def Normalize(x, means, stds):
    return (x-means)/stds

train_dataset2 = train_dataset[:,:,1:7].reshape((-1, 6)).astype(np.float32)
train_output = train_dataset[:,:,8].reshape((-1, 1)).astype(np.float32)

# calculate means and stds for training dataset
input_means = [np.mean(train_dataset2[:,i]) for i in range(train_dataset2.shape[1])]
input_stds = [np.std(train_dataset2[:,i]) for i in range(train_dataset2.shape[1])]
output_means = [np.mean(train_output[:,i]) for i in range(train_output.shape[1])]
output_stds = [np.std(train_output[:,i]) for i in range(train_output.shape[1])]

train_dataset2 = Normalize(train_dataset2, input_means, input_stds)
train_output = Normalize(train_output, output_means, output_stds)

print(train_dataset2)
print(train_output)

valid_dataset2 = Normalize(valid_dataset[:,:,1:7].reshape((-1, 6)).astype(np.float32), input_means, input_stds)
valid_output = Normalize(valid_dataset[:,:,8].reshape((-1, 1)).astype(np.float32), output_means, output_stds)

test_dataset2 = Normalize(test_dataset[:,:,1:7].reshape((-1, 6)).astype(np.float32),input_means, input_stds)
test_output = Normalize(test_dataset[:,:,8].reshape((-1, 1)).astype(np.float32), output_means, output_stds)


[[-1.72309959  2.19167495 -1.45767331 -0.70275855  0.16954818  1.37838936]
 [-1.70515358  2.17307925 -1.44289684 -0.70275855  0.17393139  1.37838936]
 [-1.68719876  2.1544857  -1.42812049 -0.70275855  0.1783146   1.37838936]
 ..., 
 [ 1.68719876  0.33845499  1.64140666  2.06858802 -0.29178849 -1.44218218]
 [ 1.70515358  0.33845499  1.64140666  2.06858802 -0.29178849 -1.44218218]
 [ 1.72309959  0.33845499  1.64140666  2.06858802 -0.29178849 -1.44218218]]
[[-0.23487496]
 [-0.23796004]
 [-0.23024735]
 ..., 
 [-0.57797915]
 [-0.57797915]
 [-0.57797915]]


In [490]:
hidden_nodes = 15

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset2)
    tf_train_labels = tf.constant(train_output)
    tf_valid_dataset = tf.constant(valid_dataset2)
    tf_test_dataset = tf.constant(test_dataset2)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights_0 = tf.Variable(tf.truncated_normal([6,hidden_nodes]))
    biases_0 = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights_h1 = tf.Variable(tf.truncated_normal([hidden_nodes,1]))
    biases_h1 = tf.Variable(tf.zeros([1]))
  
    input_layer_output = tf.matmul(tf_train_dataset, weights_0) + biases_0
    input_layer_output = tf.nn.sigmoid(input_layer_output)
    hidden_layer_output = tf.matmul(input_layer_output, weights_h1) + biases_h1
    
    loss = tf.sqrt(tf.reduce_mean(tf.square(hidden_layer_output-tf_train_labels)))
    #loss = tf.reduce_max(tf.abs(hidden_layer_output-tf_train_labels))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    #optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 10000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = loss
    valid_prediction = tf.nn.sigmoid(tf.matmul(tf_valid_dataset, weights_0) + biases_0)
    valid_prediction = tf.matmul(valid_prediction, weights_h1) + biases_h1
    test_prediction = tf.nn.sigmoid(tf.matmul(tf_test_dataset, weights_0) + biases_0)
    test_prediction = tf.matmul(test_prediction, weights_h1) + biases_h1

In [491]:
num_steps = 10001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 500 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training RMSE: %.4f' % accuracy_rmse(predictions, train_output))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation RMSE: %.4f' % accuracy_rmse(valid_prediction.eval(), valid_output))
    print('Test RMSE: %.4f' % accuracy_rmse(test_prediction.eval(), test_output))
    predicted_vs_actual = np.hstack((test_prediction.eval(), test_output))

Initialized
Loss at step 0: 1.544068
Training RMSE: 1.8396
Validation RMSE: 1.2753
Loss at step 500: 0.845279
Training RMSE: 1.3094
Validation RMSE: 0.8398
Loss at step 1000: 0.757325
Training RMSE: 1.2544
Validation RMSE: 0.7503
Loss at step 1500: 0.688979
Training RMSE: 1.2144
Validation RMSE: 0.6897
Loss at step 2000: 0.648749
Training RMSE: 1.1920
Validation RMSE: 0.6526
Loss at step 2500: 0.623075
Training RMSE: 1.1782
Validation RMSE: 0.6275
Loss at step 3000: 0.607998
Training RMSE: 1.1703
Validation RMSE: 0.6124
Loss at step 3500: 0.598226
Training RMSE: 1.1653
Validation RMSE: 0.6024
Loss at step 4000: 0.591718
Training RMSE: 1.1620
Validation RMSE: 0.5960
Loss at step 4500: 0.587799
Training RMSE: 1.1600
Validation RMSE: 0.5930
Loss at step 5000: 0.585348
Training RMSE: 1.1587
Validation RMSE: 0.5926
Loss at step 5500: 0.582650
Training RMSE: 1.1574
Validation RMSE: 0.5927
Loss at step 6000: 0.578972
Training RMSE: 1.1555
Validation RMSE: 0.5918
Loss at step 6500: 0.574953
Tr

In [508]:
print(np.corrcoef(predicted_vs_actual[:,0],predicted_vs_actual[:,1]))
print(predicted_vs_actual.shape)
plt.plot(predicted_vs_actual[:1000,0],label="predicted")
plt.plot(predicted_vs_actual[:1000,1],label="actual")
plt.ylabel("normalized output")
plt.legend(loc='upper right', shadow=True)
plt.show()

[[ 1.          0.85164894]
 [ 0.85164894  1.        ]]
(9264, 2)
